In [1]:
import json
import gzip
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
with open("../data/sentiment/train.json") as infile:
    data = json.load(infile)

In [3]:
df = pd.DataFrame(data=data)

In [4]:
df.head()

,text,start_char_pos,end_char_pos,important_span_text,sentiment
0,Epic Games has raised $2 billion in funding fr...,11,43,has raised $2 billion in funding,positive
1,The Halt is the best thing that’s happened to ...,16,42,best thing that’s happened,positive
2,#HelloFuture would just be augmented to #Hello...,13,41,would just be augmented to #,positive
3,A show of confidence that HBAR can stand on it...,31,63,can stand on its own as an asset,negative
4,": we want it, you need it!",0,13,": we want it,",positive


In [5]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def seq(a, b):
    return [x for x in range(len(a) + 1) if a[x:x+len(b)] == b]

In [7]:
X = []

for row in df.itertuples():
    encoded = tokenizer(row.text)
    txt = encoded["input_ids"]
    y = [0]
    l = 0
    
    for token in txt[1:-1]:
        l += len(tokenizer.decode(token))
        
        if row.start_char_pos <= l <= row.end_char_pos + 1:
            y.append(1)
        else:
            y.append(0)
        
    y.append(0)
    encoded["labels"] = y
    X.append(encoded)

2022-04-24 21:29:28.275283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-24 21:29:28.275301: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
X[0]

{'input_ids': [0, 39318, 636, 3100, 34, 1179, 68, 176, 325, 11, 1435, 31, 6366, 8, 5, 23863, 284, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}

In [9]:
from datasets import Dataset

In [10]:
from transformers import DataCollatorForTokenClassification

In [11]:
from transformers import create_optimizer
from transformers import TFAutoModelForTokenClassification
import tensorflow as tf

In [12]:
ds = Dataset.from_pandas(pd.DataFrame(X))

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [14]:
tf_train_set = ds.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

/home/evaldas/miniconda3/envs/rnd/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
2022-04-24 21:29:32.566199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-24 21:29:32.566218: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-24 21:29:32.566232: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eka-thinkpad): /proc/driver/nvidia/version does not exist
2022-04-24 21:29:32.566455: I tensorflow/core/pl

In [15]:
batch_size = 16
num_train_epochs = 10
num_train_steps = (len(X) // batch_size) * num_train_epochs

optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [17]:
model = TFAutoModelForTokenClassification.from_pretrained("roberta-base", num_labels=2)

All model checkpoint layers were used when initializing TFRobertaForTokenClassification.

Some layers of TFRobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [19]:
model.fit(x=tf_train_set, epochs=10)

2022-04-24 21:29:58.234367: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
625/625 [==============================] - 1555s 2s/step - loss: 0.3342
Epoch 2/10
625/625 [==============================] - 1476s 2s/step - loss: 0.2251
Epoch 3/10
625/625 [==============================] - 1475s 2s/step - loss: 0.1660
Epoch 4/10
625/625 [==============================] - 1472s 2s/step - loss: 0.1238
Epoch 5/10
625/625 [==============================] - 1479s 2s/step - loss: 0.0983
Epoch 6/10
625/625 [==============================] - 1477s 2s/step - loss: 0.0743
Epoch 7/10
625/625 [==============================] - 1513s 2s/step - loss: 0.0628
Epoch 8/10
625/625 [==============================] - 1509s 2s/step - loss: 0.0551
Epoch 9/10
625/625 [==============================] - 1479s 2s/step - loss: 0.0465
Epoch 10/10
625/625 [==============================] - 1481s 2s/step - loss: 0.0412


In [20]:
with open("../data/sentiment/testing.json") as infile:
    data = json.load(infile)

In [24]:
from transformers import pipeline

In [25]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [26]:
data[0]["text"]

'Analysts say investors should buy energy stocks because they are still cheap .'

In [27]:
import tqdm

In [28]:
results = []

for row in tqdm.tqdm(data, total=len(data)):
    tokens = tokenizer(row["text"])["input_ids"][1:-1]
    pred = nlp(row["text"])
    
    positions = []

    pos = 0
    sel_start = 0
    sel_end = 0
    sel_ent = None
    prev = "LABEL_0"
    for tok, p in zip(tokens, pred):
        l = len(tokenizer.decode(tok))
        
        ent = p["entity"]
        
        if ent != prev:
            if prev == "LABEL_0":
                sel_start = pos
                sel_ent = ent
                
            else:
                positions.append((sel_start, pos, prev))
        
        #if p["entity"] == "LABEL_0":
        #    continue
            
        #print(tok, p["entity"])
        pos += l
        prev = ent
        
    if ent != "LABEL_0":
        positions.append((sel_start, pos, ent))
        
    if len(positions) < 1:
        results.append((0, 0, 0))
    else:
        results.append(max(positions, key=lambda x: x[1] - x[0]))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [19:41<00:00,  8.46it/s]


In [29]:
res = pd.DataFrame(data=results, columns=["start", "end", "label"])
res.fillna(0, inplace=True)

In [30]:
res["text"] = [x["text"] for x in data]

In [31]:
res

,start,end,label,text
0,55,77,LABEL_1,Analysts say investors should buy energy stock...
1,3,25,LABEL_1,"Oil prices will stay high this year, given the..."
2,5,54,LABEL_1,There are chronic shortages related to underin...
3,17,32,LABEL_1,The energy sector is up over 30% this year .
4,32,67,LABEL_1,"Berkshire Hathaway BRK.B, +2.64% bought anothe..."
...,...,...,...,...
9995,5,11,LABEL_1,Just don't be the slowest gazelle .
9996,17,45,LABEL_1,Antero Resources is rated as an Industry Buy ...
9997,127,156,LABEL_1,The specific formulation of each metric is det...
9998,16,44,LABEL_1,Antero Resources is rated as an Industry Buy i...


In [32]:
submission = res.reset_index().rename(columns={"index": "id"})[["id", "start", "end"]]

In [33]:
submission.to_csv("../predictions/t2_roberta_v2.csv", index=False)

In [34]:
model.save("../models/t2_roberta_v2")

2022-04-25 01:58:22.770781: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/t2_roberta_v2/assets


INFO:tensorflow:Assets written to: ../models/t2_roberta_v2/assets
